In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
%matplotlib inline
import numpy as np
import scipy as sp
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import IPython as ip
mpl.style.use('ggplot')
mpl.rc('font', family='Noto Sans CJK TC')
ip.display.set_matplotlib_formats('svg')

In [3]:
loc = 170
scale = 5

In [4]:
alpha = 0.05  # = type I error rate = false positive rate = P(PP|AN)
beta = 0.20  # = type II error rate = false negative rate = P(PN|AP)
# confidence_level = 0.95  # = true negative rate = specificity = P(PN|AN) = 1-alpha
# power = 0.80  # = true positive rate = sensitivity = recall = P(PP|AP) = 1-beta
raw_effect_size = 2
sample_size_1 = 100
sample_size_2 = 100

## Using Solver in StatsModels

### Calculate Sample Size

In [5]:
sm.stats.tt_ind_solve_power(
    alpha=alpha,
    power=1-beta,
    # standardized effect size
    # see also: https://en.wikipedia.org/wiki/Effect_size#Cohen's_d
    effect_size=raw_effect_size/scale,
    # = sample_size_2 / sample_size_1
    ratio=1,
    nobs1=None,
)

99.08032683981143

### Calculate Effect Size

In [6]:
sm.stats.tt_ind_solve_power(
    alpha=alpha,
    power=1-beta,
    effect_size=None,
    nobs1=sample_size_1,
    ratio=sample_size_2/sample_size_1,
)*scale

1.9906955869556378

### Calculate Beta

In [7]:
1-sm.stats.tt_ind_solve_power(
    alpha=alpha,
    effect_size=raw_effect_size/scale,
    nobs1=sample_size_1,
    ratio=sample_size_2/sample_size_1,
    power=None,
)

0.19635250345692312

## Using Simulation

In [8]:
simulation_n = 1000

### Calculate Beta

In [9]:
np.random.seed(20180702)
sample_1 = sp.stats.norm.rvs(loc=loc, scale=scale, size=(sample_size_1, simulation_n))
sample_2 = sp.stats.norm.rvs(loc=loc+raw_effect_size, scale=scale, size=(sample_size_2, simulation_n))
# beta = P(PN|AP) = P(predicted negative | actual positive)
observed_beta = (sp.stats.ttest_ind(sample_1, sample_2).pvalue >= alpha).sum() / simulation_n
observed_beta

0.214

### Calculate Sample Size

In [10]:
def calc_beta_given_sample_size(x):
    np.random.seed(20180702)
    sample_1 = sp.stats.norm.rvs(loc=loc, scale=scale, size=(int(x), simulation_n))
    sample_2 = sp.stats.norm.rvs(loc=loc+raw_effect_size, scale=scale, size=(int(x), simulation_n))
    observed_beta = (sp.stats.ttest_ind(sample_1, sample_2).pvalue >= alpha).sum() / simulation_n
    return observed_beta

In [11]:
sp.optimize.brentq(
    lambda x: calc_beta_given_sample_size(x) - beta,
    200, 100
)

103.43859649122807

### Calculate Raw Effect Size

In [12]:
def calc_beta_given_raw_effect_size(x):
    np.random.seed(20180702)
    sample_1 = sp.stats.norm.rvs(loc=loc, scale=scale, size=(sample_size_1, simulation_n))
    sample_2 = sp.stats.norm.rvs(loc=loc+x, scale=scale, size=(sample_size_2, simulation_n))
    observed_beta = (sp.stats.ttest_ind(sample_1, sample_2).pvalue >= alpha).sum() / simulation_n
    return observed_beta

In [13]:
sp.optimize.brentq(
    lambda x: calc_beta_given_raw_effect_size(x) - beta,
    3, 0
)

2.0355119938176798

## PPV & NPV

We know alpha := P(predicted positive | acutal negative) = P(BD|AB) = B/AB.

How about P(AB|BD) = B/BD = FDR = false discovery rate?

### Bayes' Theorem

$
P(A \mid B) = \dfrac{P(B \mid A)\,P(A)}{P(B)} \\
\equiv \\
P(AB \mid BD) = \dfrac{P(BD \mid AB)\,P(AB)}{P(BD)} \\
\equiv \\
\text{FDR} = \dfrac{ \alpha\,P(\text{actual negative}) }{P(\text{predicted positive})} \\
$

In [14]:
# define

alpha = 0.05
beta = 0.05
p_an = 0.94

# calc

p_ap = 1-p_an
power = 1-beta
cl = 1-alpha

p_pp = alpha*p_an + power*p_ap
p_pn = cl*p_an + beta*p_ap

fdr = alpha*p_an / p_pp
for_ = beta*p_ap / p_pn
fdr, for_

(0.4519230769230767, 0.003348214285714289)